<a href="https://colab.research.google.com/github/iam-fern/Research/blob/main/ex_sampling_1000rep_n50_x8_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from sklearn.linear_model import LinearRegression
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_1000rep_n50_x8_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...         y_hat         e_hat
rep   index                        ...                            
0.0   0.0    26.154567  10.209535  ...  3.227830e+10 -9.479326e+08
      1.0    38.382103  32.032779  ...  1.059059e+11 -4.212166e+12
      2.0    18.324124  23.444792  ... -3.317305e+10  5.016796e+10
      3.0    10.000008  49.305944  ... -1.209984e+11  1.210491e+11
      4.0    43.781407  42.580957  ...  7.868071e+10  1.667081e+13
...                ...        ...  ...           ...           ...
999.0 45.0   43.546431  17.782354  ...  1.539471e+11  1.351749e+13
      46.0   16.069590  43.882442  ... -8.983237e+10  9.317856e+10
      47.0   22.722670  46.040550  ... -7.007426e+10  1.293532e+11
      48.0   47.783214  27.989152  ...  1.420384e+11 -1.997205e+13
      49.0   19.207561  46.395556  ... -3.718543e+10  4.897975e+10

[50000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...         y_hat         e_hat
rep   index                        ...                            
0.0   0.0    26.154567  10.209535  ...  3.227830e+10 -9.479326e+08
      1.0    38.382103  32.032779  ...  1.059059e+11 -4.212166e+12
      2.0    18.324124  23.444792  ... -3.317305e+10  5.016796e+10
      3.0    10.000008  49.305944  ... -1.209984e+11  1.210491e+11
      4.0    43.781407  42.580957  ...  7.868071e+10  1.667081e+13
...                ...        ...  ...           ...           ...
999.0 45.0   43.546431  17.782354  ...  1.539471e+11  1.351749e+13
      46.0   16.069590  43.882442  ... -8.983237e+10  9.317856e+10
      47.0   22.722670  46.040550  ... -7.007426e+10  1.293532e+11
      48.0   47.783214  27.989152  ...  1.420384e+11 -1.997205e+13
      49.0   19.207561  46.395556  ... -3.718543e+10  4.897975e+10

[50000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,26.154567,10.209535,48.153088,3.133036e+10,3.133036e+10,3.227830e+10,-9.479326e+08
1,0.0,1.0,38.382103,32.032779,12.521225,-4.106260e+12,-4.106260e+12,1.059059e+11,-4.212166e+12
2,0.0,2.0,18.324124,23.444792,44.631320,1.699491e+10,1.699491e+10,-3.317305e+10,5.016796e+10
3,0.0,3.0,10.000008,49.305944,35.024324,5.068413e+07,5.068432e+07,-1.209984e+11,1.210491e+11
4,0.0,4.0,43.781407,42.580957,27.715482,1.674949e+13,1.674949e+13,7.868071e+10,1.667081e+13
...,...,...,...,...,...,...,...,...,...
49995,999.0,45.0,43.546431,17.782354,19.563966,1.367144e+13,1.367144e+13,1.539471e+11,1.351749e+13
49996,999.0,46.0,16.069590,43.882442,41.851360,3.346198e+09,3.346198e+09,-8.983237e+10,9.317856e+10
49997,999.0,47.0,22.722670,46.040550,46.308728,5.927896e+10,5.927896e+10,-7.007426e+10,1.293532e+11
49998,999.0,48.0,47.783214,27.989152,23.833518,-1.983001e+13,-1.983001e+13,1.420384e+11,-1.997205e+13


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...         y_hat         e_hat
rep   index                        ...                            
0.0   0.0    26.154567  10.209535  ...  3.227830e+10 -9.479326e+08
      1.0    38.382103  32.032779  ...  1.059059e+11 -4.212166e+12
      2.0    18.324124  23.444792  ... -3.317305e+10  5.016796e+10
      3.0    10.000008  49.305944  ... -1.209984e+11  1.210491e+11
      4.0    43.781407  42.580957  ...  7.868071e+10  1.667081e+13
...                ...        ...  ...           ...           ...
999.0 45.0   43.546431  17.782354  ...  1.539471e+11  1.351749e+13
      46.0   16.069590  43.882442  ... -8.983237e+10  9.317856e+10
      47.0   22.722670  46.040550  ... -7.007426e+10  1.293532e+11
      48.0   47.783214  27.989152  ...  1.420384e+11 -1.997205e+13
      49.0   19.207561  46.395556  ... -3.718543e+10  4.897975e+10

[50000 rows x 7 columns]

In [7]:
def get_result(run_replicate, percent_data):
    new_table = table.loc[run_replicate]

    # เรียง data จากน้อยไปมากตาม x1
    new_table_sort = new_table.sort_values('x1')

    new_table_sort.reset_index(inplace=True)

    new_table_sort.drop(['index', 'y_hat', 'e_hat'], axis=1, inplace=True)


    # แบ่งข้อมูลกลุ่มที่ 1
    table1 = new_table_sort.iloc[:int(percent_data*new_table_sort.shape[0]) , :]
    y = table1['Y']
    x = table1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    # หาค่า df กลุ่ม 1
    DF1 = model1.df_resid
    # หาค่า SSE1 : Residual sum of squares
    SSE1 = model1.ssr
    # หาค่า MSE1
    MSE1 = model1.mse_resid


    # แบ่งข้อมูลกลุ่มที่ 2
    table2 = new_table_sort.iloc[int((1-percent_data)*new_table_sort.shape[0]): , :]

    y = table2['Y']
    x = table2[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model2 = sm.OLS(y, x).fit()
    #print(model.summary())

    # หาค่า df กลุ่ม 2
    DF2 = model2.df_resid
    # หาค่า SSE2 : : Residual sum of squares
    SSE2 = model2.ssr
    # หาค่า MSE2
    MSE2 = model2.mse_resid

    # ค่าสถิติทดสอบ
    GQ_test = MSE2 / MSE1


    df2 = DF2
    df1 = DF1

    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test, df2, df1)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test, df2, df1)
    
    return df2, df1, SSE2, SSE1, MSE2, MSE1, GQ_test, two_tailed, p_greater
    

    # เปรียบเทียบค่า p-value กับค่า alpha
    # Homoscedasticity : 0
    # Heteroscedasticity : 1
    #if p_value < alpha:
        #result_test='Reject005=0 : Heteroscedasticity '
        #result_test01=1
    #else:
        #result_test='Reject005=1 : Homoscedasticity '
        #result_test01=0
    
    #return df1, df2, SSE1, SSE2, MSE1, MSE2, GQ_test, result_test, p_value, result_test01


# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df1=[]
list_df2=[]
list_SSE1=[]
list_SSE2=[]
list_MSE1=[]
list_MSE2=[]
list_GQ_test=[]
#list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
#list_result_test01=[]


# กำหนด % การแบ่งข้อมูล
percent_data=0.4

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    #DF1, DF2, SSE1, SSE2, M_SSE1, M_SSE2, GQ_test, result_test=get_result(run_replicate, percent_data)
    df1, df2, SSE1, SSE2, MSE1, MSE2, GQ_test, two_tailed, p_greater=get_result(run_replicate, percent_data)
    list_df1.append(df1)
    list_SSE1.append(SSE1)
    list_df2.append(df2)
    list_SSE2.append(SSE2)
    list_MSE1.append(MSE1)
    list_MSE2.append(MSE2)
    list_GQ_test.append(GQ_test)
    #list_result_test.append(result_test)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)
    #list_result_test01.append(result_test01)

In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 50000)
result=pd.DataFrame({"Replicate":list_replicate, "df1":list_df1, "df2":list_df2, "SSE1":list_SSE1,
                     "SSE2":list_SSE2, "MSE1":list_MSE1, "MSE2":list_MSE2,
                     "GQ_test":list_GQ_test, "two_tailed":list_two_tailed, "greater":list_p_greater})
result.set_index(['Replicate'], inplace=True)
result=result[(result["two_tailed"]<=1) & (result["greater"]<=1)]
result["Result_test"]=result["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")

result.astype(str)

,df1,df2,SSE1,SSE2,MSE1,MSE2,GQ_test,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
0.0,16.0,16.0,5.099122428434666e+27,3.1435515431132305e+22,3.186951517771666e+26,1.964719714445769e+21,162208.9651943396,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
1.0,16.0,16.0,2.9623173364649414e+27,1.391779009999538e+23,1.8514483352905884e+26,8.698618812497113e+21,21284.39439869067,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
2.0,16.0,16.0,3.530281056902369e+27,2.902438836402412e+22,2.2064256605639806e+26,1.8140242727515075e+21,121631.54009054574,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
3.0,16.0,16.0,2.7405995047671226e+27,1.3526350748458018e+23,1.7128746904794516e+26,8.453969217786262e+21,20261.18910955748,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
4.0,16.0,16.0,2.5504767014301676e+27,9.36435054745534e+22,1.5940479383938547e+26,5.852719092159588e+21,27236.023347323662,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
...,...,...,...,...,...,...,...,...,...,...
995.0,16.0,16.0,4.1355950873649975e+27,1.8611178305511913e+22,2.5847469296031234e+26,1.1631986440944946e+21,222210.27704302815,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
996.0,16.0,16.0,4.645627251763116e+27,4.481153106790217e+21,2.9035170323519476e+26,2.8007206917438856e+20,1036703.5316699343,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
997.0,16.0,16.0,5.155565536635529e+27,1.2275959870861e+21,3.2222284603972055e+26,7.672474919288124e+19,4199724.983520929,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 50000)
result=pd.DataFrame({"Replicate":list_replicate, "df1":list_df1, "df2":list_df2, "SSE1":list_SSE1,
                     "SSE2":list_SSE2, "MSE1":list_MSE1, "MSE2":list_MSE2,
                     "GQ_test":list_GQ_test, "two_tailed":list_two_tailed, "greater":list_p_greater})
result.set_index(['Replicate'], inplace=True)

result['two_tailed_new']=result[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
result["Result_test"]=result["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
result_show=result.drop(columns=["two_tailed", "greater"])
result_show.astype(str)

,df1,df2,SSE1,SSE2,MSE1,MSE2,GQ_test,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,16.0,16.0,5.099122428434666e+27,3.1435515431132305e+22,3.186951517771666e+26,1.964719714445769e+21,162208.9651943396,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
1.0,16.0,16.0,2.9623173364649414e+27,1.391779009999538e+23,1.8514483352905884e+26,8.698618812497113e+21,21284.39439869067,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
2.0,16.0,16.0,3.530281056902369e+27,2.902438836402412e+22,2.2064256605639806e+26,1.8140242727515075e+21,121631.54009054574,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
3.0,16.0,16.0,2.7405995047671226e+27,1.3526350748458018e+23,1.7128746904794516e+26,8.453969217786262e+21,20261.18910955748,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
4.0,16.0,16.0,2.5504767014301676e+27,9.36435054745534e+22,1.5940479383938547e+26,5.852719092159588e+21,27236.023347323662,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
...,...,...,...,...,...,...,...,...,...
995.0,16.0,16.0,4.1355950873649975e+27,1.8611178305511913e+22,2.5847469296031234e+26,1.1631986440944946e+21,222210.27704302815,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
996.0,16.0,16.0,4.645627251763116e+27,4.481153106790217e+21,2.9035170323519476e+26,2.8007206917438856e+20,1036703.5316699343,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
997.0,16.0,16.0,5.155565536635529e+27,1.2275959870861e+21,3.2222284603972055e+26,7.672474919288124e+19,4199724.983520929,2.220446049250313e-16,Reject005=0 : Heteroscedasticity


In [10]:
result_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    1000
Name: Result_test, dtype: int64